In [ ]:
!pip install transformers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score




import torch
torch.cuda.empty_cache()
import seaborn as sns
import transformers
import json
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from transformers import RobertaModel, RobertaTokenizer
import logging
logging.basicConfig(level=logging.ERROR)
from google.colab import output
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

#GPU usage setup
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
import random
random.seed(1)
np.random.seed(1)
torch.cuda.manual_seed(1)
torch.manual_seed(1)
import time
start_time = time.time()

In [ ]:
# mount drive to access data
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:

%cd /content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics

%ls

/content/drive/MyDrive/nvdrs_narratives/round_1_annotation_topics
 breakup_feature_contributions_plot.png
 breakup_feature_importance_plot.png
 breakup_lr_model.pkl
 breakup_sample_0_sd_completed.csv
 breakup_sampled_sd_completed.csv
 child_custody_loss_feature_contributions_plot.png
 child_custody_loss_feature_importance_plot.png
 child_custody_loss_lr_model.pkl
 child_feature_contributions_plot.png
 child_feature_importance_plot.png
 child_sample_0_sd_completed.csv
 child_sampled_dw_completed.csv
 divorce_feature_contributions_plot.png
 divorce_feature_importance_plot.png
 divorce_lr_model.pkl
 divorce_sample_0_dw_completed.csv
 divorce_sampled_sr_completed.csv
 eviction_feature_contributions_plot.png
 eviction_feature_importance_plot.png
 eviction_move_roberta.pth
 eviction_or_move_lr_model.pkl
'eviction or move_sample_0_sp_completed.csv'
'eviction or move_sampled_sp_completed.csv'
 nvdrs_breakup_model_results.csv
 nvdrs_breakup_model_results.gsheet
'nvdrs_child custody loss_model_r

In [ ]:
# Read the Sample first round csv file
#df = pd.read_csv("pet_sampled_dw_completed.csv")

df1 = pd.read_csv("child_sample_0_sd_completed.csv")
df2 = pd.read_csv("child_sampled_dw_completed.csv")
# Bind/concatenate rows from additional sample, make sure we preserve column names
df = pd.concat([df1, df2])

df.info()
topic_name = "child support loss"

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100 entries, 0 to 49
Data columns (total 10 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   IncidentID                  100 non-null    int64 
 1   NarrativeCME                50 non-null     object
 2   matched_term_y              76 non-null     object
 3   CME_CircumstancesOtherText  6 non-null      object
 4   NarrativeLE                 68 non-null     object
 5   matched_term_x              82 non-null     object
 6   LE_CircumstancesOtherText   11 non-null     object
 7   topic_name                  100 non-null    object
 8   topic_relevant_annotation   100 non-null    int64 
 9   comments                    25 non-null     object
dtypes: int64(2), object(8)
memory usage: 8.6+ KB


In [ ]:
df['label'] = df['topic_relevant_annotation']
df.drop(['IncidentID','LE_CircumstancesOtherText','CME_CircumstancesOtherText','topic_name','comments'], axis=1)
del df['IncidentID'], df['LE_CircumstancesOtherText'],df['CME_CircumstancesOtherText'], df['topic_name'], df['comments'], df['topic_relevant_annotation']

In [ ]:
import pandas as pd
import numpy as np

# Assuming df is your existing DataFrame
# df = pd.DataFrame(...)  # your original DataFrame

def transform_dataframe(original_df):
    # Create an empty DataFrame with the desired structure
    transformed_df = pd.DataFrame(columns=['text', 'matched_term', 'label'])

    # Iterate over each row in the original DataFrame
    for idx, row in original_df.iterrows():
        # Logic for handling NarrativeLE and NarrativeCME
        if pd.isna(row['NarrativeLE']) or row['NarrativeLE'] == '':
            transformed_df = transformed_df.append({
                'text': row['NarrativeCME'],
                'matched_term': row['matched_term_y'],
                'label': row['label']
            }, ignore_index=True)
        elif pd.isna(row['NarrativeCME']) or row['NarrativeCME'] == '':
            transformed_df = transformed_df.append({
                'text': row['NarrativeLE'],
                'matched_term': row['matched_term_x'],
                'label': row['label']
            }, ignore_index=True)
        else:
            # Create two new rows if both columns have values
            transformed_df = transformed_df.append([
                {
                    'text': row['NarrativeLE'],
                    'matched_term': row['matched_term_x'],
                    'label': row['label']
                },
                {
                    'text': row['NarrativeCME'],
                    'matched_term': row['matched_term_y'],
                    'label': row['label']
                }
            ], ignore_index=True)

    return transformed_df

# Apply the transformation
transformed_df = transform_dataframe(df)

# Output the transformed DataFrame
transformed_df

<ipython-input-6-f20a6d615aa2>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_df = transformed_df.append({
<ipython-input-6-f20a6d615aa2>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_df = transformed_df.append({
<ipython-input-6-f20a6d615aa2>:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_df = transformed_df.append({
<ipython-input-6-f20a6d615aa2>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  transformed_df = transformed_df.append([
<ipython-input-6-f20a6d615aa2>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.conc

,text,matched_term,label
0,Male victim died of a suspected drug overdose ...,['protective custody'],0
1,On 10/25/2015 officers responded to a suicidal...,['custody'],0
2,On 3/6/2015 Officers responded to a check welf...,['custody'],0
3,"Victim, a 15-year-old white male, hung himself...",['custody'],1
4,"Victim, a 15-year-old white male, hung himself...",['custody'],1
...,...,...,...
113,Police were summoned when friends could not co...,['custody'],0
114,Victim is a 37 year old white male. He was fou...,['see his children'],1
115,"V was a 27 year old, non-Hispanic, white, male...",['see his children'],1
116,The Victim (V) was a 25 year old white male in...,['custody'],0


In [ ]:
df = transformed_df
cols = ['text', 'matched_term']
df['text'] = df[cols].apply(lambda row: '</s>'.join(row.values.astype(str)), axis=1)
df["label"] = df["label"].astype(int)
df

,text,matched_term,label
0,Male victim died of a suspected drug overdose ...,['protective custody'],0
1,On 10/25/2015 officers responded to a suicidal...,['custody'],0
2,On 3/6/2015 Officers responded to a check welf...,['custody'],0
3,"Victim, a 15-year-old white male, hung himself...",['custody'],1
4,"Victim, a 15-year-old white male, hung himself...",['custody'],1
...,...,...,...
113,Police were summoned when friends could not co...,['custody'],0
114,Victim is a 37 year old white male. He was fou...,['see his children'],1
115,"V was a 27 year old, non-Hispanic, white, male...",['see his children'],1
116,The Victim (V) was a 25 year old white male in...,['custody'],0


In [ ]:
MAX_LEN = 128
TRAIN_BATCH_SIZE = 16
VALID_BATCH_SIZE = 16
EPOCHS = 10
LEARNING_RATE = 1e-05
tokenizer = RobertaTokenizer.from_pretrained('roberta-base', truncation=True, do_lower_case=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
from torch.utils.data import Dataset
import torch

class BiasData(Dataset):
    def __init__(self, dataframe, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.text = dataframe.text
        self.targets = self.data.label
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = str(self.text[index])
        text = " ".join(text.split())

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True,
            truncation=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs.get("token_type_ids", [0] * len(ids))  # Defaulting token_type_ids to zeros if not returned

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.long)  # Ensured dtype is torch.long
        }

In [ ]:
newdf = df[['text','label']]
newdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118 entries, 0 to 117
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    118 non-null    object
 1   label   118 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.0+ KB


In [ ]:
train_size = 0.8
train_data=newdf.sample(frac=train_size,random_state=0)
test_data=newdf.drop(train_data.index).reset_index(drop=True)
train_data = train_data.reset_index(drop=True)

print("FULL Dataset: {}".format(newdf.shape))
print("TRAIN Dataset: {}".format(train_data.shape))
print("TEST Dataset: {}".format(test_data.shape))

training_set = BiasData(train_data, tokenizer, MAX_LEN)
testing_set = BiasData(test_data, tokenizer, MAX_LEN)

FULL Dataset: (118, 2)
TRAIN Dataset: (94, 2)
TEST Dataset: (24, 2)


In [ ]:
train_data['label'].unique()

array([0, 1])

In [ ]:
test_data['label'].unique()

array([0, 1])

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig.from_pretrained("roberta-base")
config.output_attentions = True

class RobertaClass(torch.nn.Module):
    def __init__(self):
        super(RobertaClass, self).__init__()
        self.l1 = RobertaModel.from_pretrained("roberta-base", config=config)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.0)
        self.classifier = torch.nn.Linear(768, 2)

    def forward(self, input_ids, attention_mask):
        # Get the hidden states, pooler output, and attention weights
        last_hidden_state, pooler_output, all_attentions = self.l1(input_ids=input_ids, attention_mask=attention_mask, return_dict=False)
        pooler = last_hidden_state[:, 0]
        pooler = self.pre_classifier(pooler)
        pooler = torch.nn.ReLU()(pooler)
        pooler = self.dropout(pooler)
        output = self.classifier(pooler)
        return output, all_attentions



In [ ]:
model = RobertaClass()
model.to(device)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaClass(
  (l1): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((

In [ ]:
# Creating the loss function and optimizer
loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def train(epoch):
    tr_loss = 0
    n_correct = 0
    nb_tr_steps = 0
    nb_tr_examples = 0
    model.train()
    for _,data in tqdm(enumerate(training_loader, 0)):
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.long)

        # Extract the logits (output) from the returned tuple
        logits, attention_weights = model(ids, mask)

        # Use the logits when computing the loss
        loss = loss_function(logits, targets)
        tr_loss += loss.item()
        big_val, big_idx = torch.max(logits.data, dim=1)
        n_correct += calcuate_accuracy(big_idx, targets)

        nb_tr_steps += 1
        nb_tr_examples += targets.size(0)

        if _ % 5000 == 0:
            loss_step = tr_loss / nb_tr_steps
            accu_step = (n_correct * 100) / nb_tr_examples
            print(f"Training Loss: {loss_step}")
            print(f"Training Accuracy: {accu_step}")

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Logits shape: {logits.shape}, Targets shape: {targets.shape}")
    print(f'The Total Accuracy for Epoch {epoch}: {(n_correct * 100) / nb_tr_examples}')
    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples
    print(f"Training Loss Epoch: {epoch_loss}")
    print(f"Training Accuracy Epoch: {epoch_accu}")

    return


In [ ]:
for epoch in range(EPOCHS):
    train(epoch)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Training Loss: 0.6999179124832153
Training Accuracy: 43.75


1it [00:01,  1.09s/it]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
4it [00:08,  1.72s/it]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max

Logits shape: torch.Size([14, 2]), Targets shape: torch.Size([14])
The Total Accuracy for Epoch 0: 57.4468085106383
Training Loss Epoch: 0.6894948383172353
Training Accuracy Epoch: 57.4468085106383


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  6.72it/s]

...

Logits shape: torch.Size([14, 2]), Targets shape: torch.Size([14])
The Total Accuracy for Epoch 8: 70.2127659574468
Training Loss Epoch: 0.6188414096832275
Training Accuracy Epoch: 70.2127659574468


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  6.36it/s]

Training Loss: 0.587214469909668
Training Accuracy: 75.0


3it [00:00,  2.99it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
4it [00:01,  1.72it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max

Logits shape: torch.Size([14, 2]), Targets shape: torch.Size([14])
The Total Accuracy for Epoch 9: 70.2127659574468
Training Loss Epoch: 0.597067634264628
Training Accuracy Epoch: 70.2127659574468


In [ ]:
def calcuate_accuracy(preds, targets):
    n_correct = (preds==targets).sum().item()
    return n_correct

In [ ]:
def valid(model, testing_loader):
    model.eval()
    n_correct = 0
    tr_loss = 0
    nb_tr_steps = 0
    nb_tr_examples = 0

    all_preds = []  # list to store predictions
    all_targets = []  # list to store original targets
    all_texts = []  # list to store original input texts

    with torch.no_grad():
        for _, data in tqdm(enumerate(testing_loader, 0)):
            ids = data['ids'].to(device, dtype=torch.long)
            mask = data['mask'].to(device, dtype=torch.long)
            targets = data['targets'].to(device, dtype=torch.long)

            # Extract the logits (output) from the returned tuple
            logits, attention_weights = model(ids, mask)
           # logits = logits.squeeze()
            print(f"Batch size: {len(targets)}")  # Print actual batch size
            print(f"Logits shape: {logits.shape}, Targets shape: {targets.shape}")
            # Use the logits when computing the loss and other operations
            loss = loss_function(logits, targets)
            tr_loss += loss.item()
            big_val, big_idx = torch.max(logits.data, dim=1)

            all_preds.extend(big_idx.cpu().numpy())  # store predictions
            all_targets.extend(targets.cpu().numpy())  # store targets

            all_texts.extend(data['ids'])  # store original input texts

            n_correct += calcuate_accuracy(big_idx, targets)9

            nb_tr_steps += 1
            nb_tr_examples += targets.size(0)

            if _ % 5000 == 0:
                loss_step = tr_loss / nb_tr_steps
                accu_step = (n_correct * 100) / nb_tr_examples
                print(f"Validation Loss: {loss_step}")
                print(f"Validation Accuracy: {accu_step}")

    epoch_loss = tr_loss / nb_tr_steps
    epoch_accu = (n_correct * 100) / nb_tr_examples


    print(f"Validation Loss Epoch: {epoch_loss}")
    print(f"Validation Accuracy Epoch: {epoch_accu}")

    # Print classification report
    report = classification_report(all_targets, all_preds)
    print(report)

    # Confusion matrix
    cm = confusion_matrix(all_targets, all_preds)
    print("Confusion Matrix:")
    print(cm)

    # Create a DataFrame and save it
    df_predictions = pd.DataFrame({
        'Text': all_texts,
        'Original': all_targets,
        'Predicted': all_preds
    })
    df_predictions.to_csv('predictions.csv', index=False)
    print(df_predictions.head())

    return epoch_accu



Initial Run -- one interation

In [ ]:
valid(model, testing_loader)

acc = valid(model,testing_loader)
print("test accuracy = %0.2f%%" % acc)

end_time = time.time()

elapsed_time = end_time - start_time
print(f"Elapsed Time: {elapsed_time:.2f} seconds")

1it [00:00,  3.46it/s]

Batch size: 16
Logits shape: torch.Size([16, 2]), Targets shape: torch.Size([16])
Validation Loss: 0.7173306345939636
Validation Accuracy: 43.75
Batch size: 8
Logits shape: torch.Size([8, 2]), Targets shape: torch.Size([8])


2it [00:00,  5.19it/s]


Validation Loss Epoch: 0.682496964931488
Validation Accuracy Epoch: 50.0
              precision    recall  f1-score   support

           0       0.55      0.46      0.50        13
           1       0.46      0.55      0.50        11

    accuracy                           0.50        24
   macro avg       0.50      0.50      0.50        24
weighted avg       0.51      0.50      0.50        24

Confusion Matrix:
[[6 7]
 [5 6]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(133), tensor(468), tensor(3...         1          1
1  [tensor(0), tensor(846), tensor(16), tensor(10...         0          0
2  [tensor(0), tensor(3103), tensor(1423), tensor...         1          1
3  [tensor(0), tensor(846), tensor(21), tensor(30...         0          0
4  [tensor(0), tensor(133), tensor(468), tensor(6...         0          1


0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
1it [00:00,  7.96it/s]

Batch size: 16
Logits shape: torch.Size([16, 2]), Targets shape: torch.Size([16])
Validation Loss: 0.7078904509544373
Validation Accuracy: 50.0


2it [00:00,  9.86it/s]

Batch size: 8
Logits shape: torch.Size([8, 2]), Targets shape: torch.Size([8])
Validation Loss Epoch: 0.6872171759605408
Validation Accuracy Epoch: 50.0
              precision    recall  f1-score   support

           0       0.55      0.46      0.50        13
           1       0.46      0.55      0.50        11

    accuracy                           0.50        24
   macro avg       0.50      0.50      0.50        24
weighted avg       0.51      0.50      0.50        24

Confusion Matrix:
[[6 7]
 [5 6]]
                                                Text  Original  Predicted
0  [tensor(0), tensor(133), tensor(1802), tensor(...         0          1
1  [tensor(0), tensor(713), tensor(1718), tensor(...         1          0
2  [tensor(0), tensor(3750), tensor(2219), tensor...         0          1
3  [tensor(0), tensor(133), tensor(468), tensor(3...         1          1
4  [tensor(0), tensor(846), tensor(134), tensor(3...         0          0
test accuracy = 50.00%
Elapsed Time: 52.45 

In order to account for variations in GPU states, we ran each of the models 10 times and provide the mean accuracy


## Predictions

In [ ]:
def predict_on_dataframe(df, model, tokenizer, max_len):
    model.eval()
    predictions = []

    for index, row in df.iterrows():
        text = row['text']
        inputs = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = torch.tensor([inputs['input_ids']], dtype=torch.long).to(device)
        mask = torch.tensor([inputs['attention_mask']], dtype=torch.long).to(device)

        with torch.no_grad():
            logits, _ = model(ids, mask)  # Here we unpack the tuple

        big_val, big_idx = torch.max(logits.data, dim=1)  # Use logits instead of outputs
        predictions.append(big_idx[0].item())

    df['predictions'] = predictions
    return df



In [ ]:
predictions = predict_on_dataframe(newdf, model, tokenizer, max_len=512)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2614: FutureWarning

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score



true_labels = predictions['label'].tolist()
predicted_classes = predictions['predictions'].tolist()

def bootstrap_metric(y_true, y_pred, metric_func, n_iterations=1000):
    bootstrap_values = []
    n_samples = len(y_true)
    for _ in range(n_iterations):
        indices = np.random.choice(np.arange(n_samples), size=n_samples, replace=True)
        bootstrap_true = np.array(y_true)[indices]
        bootstrap_pred = np.array(y_pred)[indices]
        value = metric_func(bootstrap_true, bootstrap_pred)
        bootstrap_values.append(value)
    return bootstrap_values

# Direct calculation of metrics for point estimates
point_estimate_accuracy = accuracy_score(true_labels, predicted_classes)
point_estimate_f1_pos = f1_score(true_labels, predicted_classes, pos_label=1)
point_estimate_precision_pos = precision_score(true_labels, predicted_classes, pos_label=1)
point_estimate_recall_pos = recall_score(true_labels, predicted_classes, pos_label=1)

point_estimate_f1_macro = f1_score(true_labels, predicted_classes, average='macro')
point_estimate_precision_macro = precision_score(true_labels, predicted_classes, average='macro')
point_estimate_recall_macro = recall_score(true_labels, predicted_classes, average='macro')

# Bootstrapping each metric including accuracy
bootstrap_accuracy = bootstrap_metric(true_labels, predicted_classes, accuracy_score)
bootstrap_f1_pos = bootstrap_metric(true_labels, predicted_classes, lambda y_true, y_pred: f1_score(y_true, y_pred, pos_label=1))
bootstrap_precision_pos = bootstrap_metric(true_labels, predicted_classes, lambda y_true, y_pred: precision_score(y_true, y_pred, pos_label=1))
bootstrap_recall_pos = bootstrap_metric(true_labels, predicted_classes, lambda y_true, y_pred: recall_score(y_true, y_pred, pos_label=1))

bootstrap_f1_macro = bootstrap_metric(true_labels, predicted_classes, lambda y_true, y_pred: f1_score(y_true, y_pred, average='macro'))
bootstrap_precision_macro = bootstrap_metric(true_labels, predicted_classes, lambda y_true, y_pred: precision_score(y_true, y_pred, average='macro'))
bootstrap_recall_macro = bootstrap_metric(true_labels, predicted_classes, lambda y_true, y_pred: recall_score(y_true, y_pred, average='macro'))

# Computing 95% confidence intervals
def confidence_interval(data, alpha=0.05):
    lower_percentile = 100 * alpha / 2.
    upper_percentile = 100 * (1 - alpha / 2.)
    lower = np.percentile(data, lower_percentile)
    upper = np.percentile(data, upper_percentile)
    return lower, upper

# Printing point estimates and confidence intervals
print(f"Accuracy: {point_estimate_accuracy}, CI:", confidence_interval(bootstrap_accuracy))
print(f"Positive Class Precision: {point_estimate_precision_pos}, CI:", confidence_interval(bootstrap_precision_pos))
print(f"Positive Class Recall: {point_estimate_recall_pos}, CI:", confidence_interval(bootstrap_recall_pos))
print(f"Positive Class F1 Score: {point_estimate_f1_pos}, CI:", confidence_interval(bootstrap_f1_pos))

print(f"Macro Precision: {point_estimate_precision_macro}, CI:", confidence_interval(bootstrap_precision_macro))
print(f"Macro Recall: {point_estimate_recall_macro}, CI:", confidence_interval(bootstrap_recall_macro))
print(f"Macro F1 Score: {point_estimate_f1_macro}, CI:", confidence_interval(bootstrap_f1_macro))


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defin

Accuracy: 0.4576271186440678, CI: (0.3644067796610169, 0.5423728813559322)
Positive Class Precision: 0.6666666666666666, CI: (0.0, 1.0)
Positive Class Recall: 0.03076923076923077, CI: (0.0, 0.07936507936507936)
Positive Class F1 Score: 0.05882352941176471, CI: (0.0, 0.14036323202372128)
Macro Precision: 0.5594202898550724, CI: (0.2076271186440678, 0.7606884057971014)
Macro Recall: 0.5059506531204645, CI: (0.4787296037296037, 0.5330516820671297)
Macro F1 Score: 0.3389355742296919, CI: (0.2804539932147442, 0.4057191634656424)


# Attention Visualization

In [ ]:
# Save model
import torch

# Load model
# model is an instance of a custom class 'RobertaClass'
# Save the model's state_dict
torch.save(model, "eviction_move_roberta.pth")


In [ ]:
#import matplotlib.pyplot as plt
#import numpy as np

# We already have the tokenized input_text
#print(tokens)

# We also already have the attention tensor: attention_tensor
#attention_matrix = attention_tensor[0].detach().cpu().numpy()  # Convert tensor to numpy array

# Identify the token position for the word "needy".
#needy_position = tokens.index('Ġneedy') if 'Ġneedy' in tokens else None

# Identify non-padding tokens
#non_padding_positions = [pos for pos, token_id in enumerate(inputs['input_ids'][0].cpu().tolist()) if token_id != tokenizer.pad_token_id]

#if needy_position is not None:
    # Use matplotlib to visualize the attention weights for this token.

    # Restrict to only the non-padding tokens
#    restricted_attention = attention_matrix[0, 0, needy_position, non_padding_positions]
#    restricted_tokens = [tokens[pos] for pos in non_padding_positions]

#    plt.figure(figsize=(10, 5))
#    plt.bar(np.arange(len(restricted_tokens)), restricted_attention)
#    plt.xlabel('Tokens')
#    plt.ylabel('Attention Weight')
#    plt.title('Attention Weights for the word "needy"')
#    plt.xticks(np.arange(len(restricted_tokens)), restricted_tokens, rotation=90)
#    plt.tight_layout()
#    plt.savefig("needy_attention_diagram.png", format='png', dpi=300, bbox_inches='tight')
#    plt.show()
#else:
#    print("The word 'needy' not found in tokens.")





Notes for next time-- want to turn this script into a function where I can pass through the hyperparameters.

In [ ]:
# svm baseline

#from sklearn.pipeline import make_pipeline
#from sklearn.preprocessing import StandardScaler
#from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.feature_extraction.text import TfidfTransformer
#from sklearn.svm import SVC

#from sklearn.pipeline import Pipeline
#text_clf = Pipeline([('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),('clf', SVC(gamma='auto'))])
#text_clf.fit(train_data['text'], train_data['label'])

In [ ]:
#docs_test = test_data['text']
#predicted = text_clf.predict(docs_test)
#np.mean(predicted == test_data['label'])

In [ ]:
#test_accs = []
#for i in range(10):
#  model = RobertaClass()
#  model.to(device)
#
#  loss_function = torch.nn.CrossEntropyLoss()
#  optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)
#
#  for epoch in range(EPOCHS):
#      train(epoch)
#  acc = valid(model, testing_loader)
#  #print("test accuracy = %0.2f%%" % acc)
#  test_accs.append(acc)

# mean_acc = sum(test_accs) / len(test_accs)
# print("MEAN ACCURACY = %0.2f%%" % mean_acc)
# output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/d/d9/Wilhelm_Scream.ogg").play()')
